In [12]:
%pip install fcx-playground

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import shutil
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import zarr

from typing import Generator

from fcx_playground.fcx_dataprocess.tiles_rad_range import RadRangeTilesPointCloudDataProcess

In [14]:
class HIWRAPTilesPointCloudDataProcess(RadRangeTilesPointCloudDataProcess):
    def _cleaning(self, data: xr.Dataset) -> xr.Dataset:
      # data extraction
      # scrape necessary data columns 
      extracted_data = data[['time', 'ref', 'lat', 'lon', 'alt', 'roll', 'pitch', 'head', 'range']]
      return extracted_data

    def _transformation(self, data: xr.Dataset) -> pd.DataFrame:
      #  transform the data to a suitable data formatting
      secs = data['time'].values # seconds since 2013-09-22T00:00:00Z
      lat = data['lat'].values
      lon = data['lon'].values
      alt = data['alt'].values # altitude of aircraft in meters
      roll = data["roll"].values
      pitch = data["pitch"].values
      head = data["head"].values
      ref = data['ref'].values #CRS radar reflectivity #2d data
      rad_range = data["range"].values # has lower count than ref
      
      # time correction and conversion:
      base_time = np.datetime64('{}-{}-{}'.format("2013", "09", "22")) # all the datapoints are with respect to 2013-02-22 base point. refer documentation
      delta = secs.astype('timedelta64[s]') + base_time
      time = (delta).astype('timedelta64[s]').astype(np.int64)

      # transform ref to 1d array and repeat other columns to match data dimension

      num_col = ref.shape[0] # number of cols
      num_row = ref.shape[1] # number of rows

      time = np.repeat(time, num_row)
      lon = np.repeat(lon, num_row)
      lat = np.repeat(lat, num_row)
      alt = np.repeat(alt, num_row)
      roll = np.repeat(roll * self.to_rad, num_row)
      pitch = np.repeat(pitch * self.to_rad, num_row)
      head = np.repeat(head * self.to_rad, num_row)
      rad_range = np.tile(rad_range, num_col)
      ref = ref.flatten()

      # curtain creation

      x, y, z = self._down_vector(roll, pitch, head)
      x = np.multiply(x, np.divide(rad_range, 111000 * np.cos(lat * self.to_rad)))
      y = np.multiply(y, np.divide(rad_range, 111000))
      z = np.multiply(z, rad_range)
      lon = np.add(-x, lon)
      lat = np.add(-y, lat)
      alt = np.add(z, alt)

      # sort by time

      sort_idx = np.argsort(time)
      lon = lon[sort_idx]
      lat = lat[sort_idx]
      alt = alt[sort_idx]
      ref = ref[sort_idx]
      time = time[sort_idx]

      # remove nan and infinite using mask (dont use masks filtering for values used for curtain creation)

      mask = np.logical_and(np.isfinite(ref), alt > 0)
      time = time[mask]
      ref = ref[mask]
      lon = lon[mask]
      lat = lat[mask]
      alt = alt[mask]


      df = pd.DataFrame(data = {
        'time': time[::20],
        'lon': lon[::20],
        'lat': lat[::20],
        'alt': alt[::20],
        'ref': ref[::20]
      })
      print(df)
      return df

    def _get_date_from_url(self, url: str) -> np.datetime64:
      # get date from url
      # date is in the format of YYYYMMDD
      # eg. 20190801
      date = url.split("HS3_HIWRAP_")[1].split("_")[0]
      np_date = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))
      return np_date

In [15]:
obj = HIWRAPTilesPointCloudDataProcess()

In [16]:
# %pip install netCDF4

In [17]:
data = obj.ingest("../../../../test_data/HS3_HIWRAP_20130925_kuinnerchirp_175902-183052_v03.nc")

In [18]:
data

<xarray.Dataset>
Dimensions:       (time: 115058, range: 353)
Coordinates:
  * range         (range) float32 74.95 149.9 224.8 ... 2.638e+04 2.646e+04
  * time          (time) float32 3.239e+05 3.239e+05 ... 3.259e+05 3.259e+05
Data variables: (12/34)
    alt           (time) float32 ...
    dopcorr       (time, range) float32 ...
    doph          (time, range) float32 ...
    dophu         (time, range) float32 ...
    dopl          (time, range) float32 ...
    doplh         (time, range) float32 ...
    ...            ...
    vacft         (time) float32 ...
    vuh           float32 ...
    vul           float32 ...
    vulh          float32 ...
    wvel          (time) float32 ...
    year          int16 ...
Attributes:
    title:        HIWRAP Data, NASA Goddard Space Flight Center
    filename:     /karldata4/tian/HS3/netcdf/2013/dist/dat/v3/hs3_hiwrap_kuin...
    experiment:   NASA HS3
    source:       created from HIWRAP binary file, Version 0.0
    comments:     Contact: lin.tian-1@nasa.gov
    Conventions:  CF-1.6
    history:      Wed Sep 23 15:42:03 2015: ncks -4 -L 4 -O /ghrc/ops/output/...
    NCO:          4.3.7

In [19]:
pre_processed_data = obj.preprocess(data)

               time        lon        lat           alt         ref
0        1380131942 -73.090666  29.753222  16659.091797   37.849857
1        1380131942 -73.085731  29.755217   4080.413086    3.818015
2        1380131942 -73.086370  29.754979   5577.877930    1.033423
3        1380131942 -73.084804  29.755561   1909.089844   18.673658
4        1380131942 -73.090302  29.753532  14712.404297   24.738737
...             ...        ...        ...           ...         ...
1302694  1380133896 -72.364106  28.536864  14405.318359   17.360151
1302695  1380133896 -72.367366  28.536433   6017.289062   -0.503962
1302696  1380133896 -72.368065  28.536341   4219.853516    5.402319
1302697  1380133896 -72.366173  28.536591   9087.906250 -999.000000
1302698  1380133896 -72.366900  28.536495   7215.579102 -999.000000

[1302699 rows x 5 columns]


In [20]:
point_clouds_tileset = obj.prep_visualization(pre_processed_data)

In [21]:
point_clouds_tileset

'temp/2013-09-25/zarr_point_cloud'

Download data form s3

In [ ]:
from boto3 import client as boto_client
from pathlib import Path

In [ ]:
def downloadFromS3(bucket_name, s3_key, dest_dir):
    s3 = boto_client('s3')
    filename = s3_key.split('/')[3]
    dest = f"{dest_dir}/{filename}"
    if os.path.exists(dest_dir): shutil.rmtree(f"{dest_dir}")
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    print("Downloading file",s3_key,"from bucket",bucket_name, " into dir:", dest_dir)
    s3.download_file(
        Bucket = bucket_name,
        Key = s3_key,
        Filename = dest
    )
    return dest

In [ ]:
from botocore.exceptions import ClientError, NoCredentialsError

def upload_to_s3(file_name, bucket, key=None):
    """Upload a file to an S3 bucket
     file_name: File to upload
     bucket: S3 bucket to upload to
     object_name: S3 object name. If not specified then file_name is used
    """
    if key is None: key = file_name

    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_name, bucket, key)
    except ClientError as e:
       print(e)
    except NoCredentialsError:
        print("%%Credentials not available")


In [ ]:
def uploadFolderToS3(point_cloud_folder, bucket_name, key):
  # UPLOAD CONVERTED FILES.
  files = os.listdir(point_cloud_folder)
  for file in files:
      fname = os.path.join(point_cloud_folder, file) # SOURCE
      # key = f"{field_campaign}/{output_data_dir}/hiwrap/{sdate}/{file}" # DESTINATION
      print(f"uploaded to {key}.")
      upload_to_s3(fname, bucket_name, key)